In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
import math
nltk.download('stopwords')

data = pd.read_csv("winemag-data-130k-v2.csv")
data.drop(columns=["Unnamed: 0"],inplace=True)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gneur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = data.drop_duplicates(["description","price","points","taster_name","title"])
len(data)

119988

In [3]:
"""
missingOnlyName = 0
missingOnlyTwitter = 0
missingBoth = 0
lis = []
for idx,row in data.iterrows():
    #print(row["taster_name"])
    if(pd.isnull(row["taster_name"]) and (not pd.isnull(row["taster_twitter_handle"]))):
        missingOnlyName = missingOnlyName + 1
    if(pd.isnull(row["taster_twitter_handle"]) and (not pd.isnull(row["taster_name"]))):
        missingOnlyTwitter = missingOnlyTwitter + 1
    if(pd.isnull(row["taster_name"]) and pd.isnull(row["taster_twitter_handle"])):
        missingBoth = missingBoth + 1
    couple = [row["taster_name"],row["taster_twitter_handle"]]
    if not couple in lis:
        lis.append(couple)
print(missingOnlyName,missingOnlyTwitter,missingBoth)
print(lis,len(lis))
"""
data.drop(["taster_twitter_handle"],axis=1,inplace=True)

In [4]:
avgPriceWinery = data.groupby("winery")#.price.mean()
#avgPriceWinery.get_group("1+1=3").price.mean()
data['price'] = data.apply(
    lambda row: avgPriceWinery.get_group(row["winery"]).price.mean() if pd.isnull(row['price']) else row['price'],
    axis=1
)
data["price"].fillna(data["price"].mean(),inplace=True)

#prende solo recensioni con autore conosciuto, ha senso come cosa?
#data.dropna(subset=["taster_name"],inplace=True)

data.fillna("unknown",inplace=True)

In [5]:
actualData = data.groupby("country").filter(lambda x: len(x)>600) 
len(actualData)

117652

In [6]:
actualData.drop(["points","price","region_1","region_2","taster_name"],axis = 1,inplace=True)

C:\Users\gneur\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [7]:
actualData["word_count"] = actualData["description"].apply(lambda x: len(str(x).split(" ")))
actualData.head()

C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,country,description,designation,province,title,variety,winery,word_count
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,39
2,US,"Tart and snappy, the flavors of lime flesh and...",unknown,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41


In [8]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

actualData["stopwords"] = actualData["description"].apply(lambda x: len([x for x in x.split() if x in stop]))
actualData.head()

C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,country,description,designation,province,title,variety,winery,word_count,stopwords
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24,3
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,39,15
2,US,"Tart and snappy, the flavors of lime flesh and...",unknown,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28,8
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33,11
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41,16


In [9]:
print("average words in description " + str(actualData["word_count"].mean()))
print("average stopwords in description " + str(actualData["stopwords"].mean()))

average words in description 40.399602216706896
average stopwords in description 13.690128514602387


# pre processing

In [10]:
#lower case everything
actualData['description'] = actualData['description'].apply(lambda x: " ".join(x.lower() for x in x.split()))
actualData.head()

C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,country,description,designation,province,title,variety,winery,word_count,stopwords
0,Italy,"aromas include tropical fruit, broom, brimston...",Vulkà Bianco,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24,3
1,Portugal,"this is ripe and fruity, a wine that is smooth...",Avidagos,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,39,15
2,US,"tart and snappy, the flavors of lime flesh and...",unknown,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28,8
3,US,"pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33,11
4,US,"much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41,16


In [11]:
#remove punctuation
actualData['description'] = actualData['description'].str.replace('[^\w\s]','')
actualData.head()

C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,country,description,designation,province,title,variety,winery,word_count,stopwords
0,Italy,aromas include tropical fruit broom brimstone ...,Vulkà Bianco,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24,3
1,Portugal,this is ripe and fruity a wine that is smooth ...,Avidagos,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,39,15
2,US,tart and snappy the flavors of lime flesh and ...,unknown,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28,8
3,US,pineapple rind lemon pith and orange blossom s...,Reserve Late Harvest,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33,11
4,US,much like the regular bottling from 2012 this ...,Vintner's Reserve Wild Child Block,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41,16


In [12]:
#remove stopwords
actualData['description'] = actualData['description'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
actualData.drop(["stopwords"],axis=1,inplace=True)
actualData.head()

C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\gneur\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,country,description,designation,province,title,variety,winery,word_count
0,Italy,aromas include tropical fruit broom brimstone ...,Vulkà Bianco,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24
1,Portugal,ripe fruity wine smooth still structured firm ...,Avidagos,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,39
2,US,tart snappy flavors lime flesh rind dominate g...,unknown,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28
3,US,pineapple rind lemon pith orange blossom start...,Reserve Late Harvest,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33
4,US,much like regular bottling 2012 comes across r...,Vintner's Reserve Wild Child Block,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41


In [13]:
#remove most frequent words, not influent bcs very frequent
freq = pd.Series(' '.join(actualData['description']).split()).value_counts()[:10]
freq

wine       70938
flavors    56238
fruit      40786
aromas     35161
palate     33727
acidity    31675
finish     30864
tannins    27826
drink      27415
ripe       24531
dtype: int64

In [14]:
freq = list(freq.index)
actualData['description'] = actualData['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
actualData.head()

C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,country,description,designation,province,title,variety,winery,word_count
0,Italy,include tropical broom brimstone dried herb is...,Vulkà Bianco,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24
1,Portugal,fruity smooth still structured firm filled jui...,Avidagos,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,39
2,US,tart snappy lime flesh rind dominate green pin...,unknown,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28
3,US,pineapple rind lemon pith orange blossom start...,Reserve Late Harvest,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33
4,US,much like regular bottling 2012 comes across r...,Vintner's Reserve Wild Child Block,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41


In [15]:
actualData["word_count"] = actualData["description"].apply(lambda x: len(str(x).split(" ")))
actualData.head()
print(actualData["word_count"].mean())

21.79407064903274


C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
unfreq = pd.Series(' '.join(actualData["description"]).split()).value_counts()
count = 0
for el in unfreq: 
    if(el < 10):
        count = count + 1
print(count)   #elementi poco frequenti sotto il 10 sono spesso typos/parole unite o robe brutte

36073


In [17]:
unfreqList = unfreq[-count:]
unfreqList

fullfledged          9
footprint            9
competitors          9
welcomed             9
cabmerlot            9
subpar               9
bars                 9
lax                  9
charmingly           9
illustrates          9
legacy               9
winelike             9
difficulty           9
herbally             9
imbalance            9
upside               9
ford                 9
unlimited            9
montsant             9
pyrazinebased        9
wineits              9
grapegrowing         9
hat                  9
sabrine              9
20172020             9
bathed               9
sisters              9
400                  9
brunate              9
fewer                9
                    ..
whitecraft           1
116acre              1
fouryear             1
beerenausleselike    1
slid                 1
alexis               1
regale               1
isabellethis         1
alloyed              1
soilhence            1
cuyo                 1
tolpuddles           1
ashall     

In [18]:
unfreq = dict(unfreqList)  #con dictionary is veeeeeeeeeeeery fasteroni
actualData["description"] = actualData['description'].apply(lambda x: " ".join(x for x in x.split() if x not in unfreq))

C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
actualData["numbers_count"] = actualData["description"].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
print(actualData["numbers_count"].mean())
#forse è meglio rimuovere anche i numeri?

0.366691598952844


C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [20]:
actualData["description"] = actualData['description'].apply(lambda x: " ".join(x for x in x.split() if not x.isdigit()))

C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
#rimosso tutto il removibile? come viene adesso la situa? 
actualData["description"]

0         include tropical broom brimstone dried herb is...
1         fruity smooth still structured firm filled jui...
2         tart snappy lime flesh rind dominate green pin...
3         pineapple rind lemon pith orange blossom start...
4         much like regular bottling comes across rather...
5         blackberry raspberry show typical navarran whi...
6         heres bright informal red opens candied berry ...
7         dry restrained offers spice profusion balanced...
8         savory dried thyme notes accent preserved peac...
9         great depth flavor fresh apple pear fruits tou...
10        soft supple plum oaky structure cabernet suppo...
11        dry spicy tight taut texture strongly mineral ...
12        slightly reduced offers chalky tannic backbone...
13        dominated oak oakdriven include roasted coffee...
14        building years six generations winemaking trad...
15        zesty orange peels apple notes abound sprightl...
16        baked plum molasses balsamic v

In [22]:
#nelle descrizioni si trovano anche i nomi di altri vini, tipo "ah ha 30% di bordeaux". Rimuovere nomi dei vini o lasciare tutto?

In [23]:
actualData["word_count"] = actualData["description"].apply(lambda x: len(str(x).split(" ")))
actualData.head()
print(actualData["word_count"].mean())

20.773824499371027


C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
actualData

,country,description,designation,province,title,variety,winery,word_count,numbers_count
0,Italy,include tropical broom brimstone dried herb is...,Vulkà Bianco,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,16,0
1,Portugal,fruity smooth still structured firm filled jui...,Avidagos,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,16,1
2,US,tart snappy lime flesh rind dominate green pin...,unknown,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,13,0
3,US,pineapple rind lemon pith orange blossom start...,Reserve Late Harvest,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,17,0
4,US,much like regular bottling comes across rather...,Vintner's Reserve Wild Child Block,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,23,1
5,Spain,blackberry raspberry show typical navarran whi...,Ars In Vitro,Northern Spain,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,22,0
6,Italy,heres bright informal red opens candied berry ...,Belsito,Sicily & Sardinia,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,15,0
7,France,dry restrained offers spice profusion balanced...,unknown,Alsace,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,10,0
8,Germany,savory dried thyme notes accent preserved peac...,Shine,Rheinhessen,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel,13,0
9,France,great depth flavor fresh apple pear fruits tou...,Les Natures,Alsace,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,13,0


In [25]:
#groupby variety, for every description of that wine count the number of time a term is used. Prova a fare lo stesso pre-post 
#stemming e lemmatization.
#probabilmente se ottieni vettori "corti" e simili,con le stesse parole/stems/lemmas che appaiono spesso dovrebbe essere bene per la classificazione/predizione. 
#Word2Vector? Usare i sinonimi? migliora la situazione?

In [26]:
#usare tf-idf? "L'idea alla base di questo comportamento è di dare più importanza ai termini che compaiono nel documento, ma che in generale sono poco frequenti"
#così si dovrebbe ottenere i termini più "caratteristici" per ogni tipo di vino? 

In [27]:
fullText = pd.Series(' '.join(actualData['description']).split()).value_counts()
fullText

cherry             24057
black              22571
notes              16942
spice              16830
red                16751
rich               15865
oak                15259
fresh              15171
nose               14859
dry                14140
berry              14020
plum               12553
fruits             12332
soft               12197
blend              11680
crisp              11524
blackberry         11453
sweet              11421
apple              11360
texture            11359
offers             11179
white              10783
shows              10636
light              10374
dark               10258
citrus             10118
bright              9686
well                9585
cabernet            9548
vanilla             9538
                   ...  
internationally       10
tomatoleaf            10
curedmeat             10
bistro                10
shiny                 10
perceived             10
introducing           10
lowerpriced           10
romaneira             10


In [28]:
totalWords = pd.Series("".join(actualData["description"]).split()).value_counts().sum()
print(totalWords)

2326431


In [29]:
t1 = actualData.groupby(["variety"])
t2 = pd.DataFrame({'text' : t1['description'].apply(lambda x: (x + " ").sum())}).reset_index()

In [30]:
t2["word_count"] = t2["text"].apply(lambda x: len(str(x).split(" ")))
t2 = t2.sort_values('word_count',ascending = False)
#t2.iloc[0]["text"]
t2

,variety,text,word_count
383,Pinot Noir,much like regular bottling comes across rather...,267059
112,Chardonnay,building years six generations winemaking trad...,209731
68,Cabernet Sauvignon,soft supple plum oaky structure cabernet suppo...,188478
410,Red Blend,dark berries mingle ample notes black pepper t...,183038
52,Bordeaux-style Red Blend,blend merlot cabernet franc brings disjointed ...,120495
416,Riesling,pineapple rind lemon pith orange blossom start...,101398
446,Sauvignon Blanc,shows tart green gooseberry flavor similar new...,86042
488,Syrah,baked red cherry crushed clove iron rose petal...,84926
331,Nebbiolo,slightly particularly given vintage complex no...,63158
439,Sangiovese,densely hued black plum vanilla carry simple a...,56136


In [31]:
varietyDict = {}
for idx, row in t2.iterrows():
    varietyDict[row["variety"]] = pd.Series(row['text'].split()).value_counts()
#varietyDict["Pinot Noir"]

In [32]:
#tf per variety
tfDict = {}
for var in varietyDict:
    temp = {}
    for word in varietyDict[var].keys():
        temp[word] = varietyDict[var][word] / fullText[word]   #compute tf= numero di occorrenze della parola nel documento /numero di occorrenze della stessa parola in tutti i documenti 
    tfDict[var] = temp

In [33]:
#idf per variety
idfDict = {}
N = len(t2)                                #numero di documenti
idfDict = dict.fromkeys(fullText.keys(),0) #dizionario contenente tutte le parole presenti 
for var in varietyDict:
    for word in varietyDict[var].keys():
        idfDict[word] = math.log10(N/fullText[word])
idfDict

{'cherry': -1.5916607560314366,
 'black': -1.5639700885996959,
 'notes': -1.4393839652407352,
 'spice': -1.4365034038113984,
 'red': -1.4344600264632688,
 'rich': -1.4108593642720055,
 'oak': -1.393945360857296,
 'fresh': -1.3914334961877814,
 'nose': -1.3824088705376931,
 'dry': -1.3608686972964552,
 'berry': -1.3571673014662144,
 'plum': -1.3091668166603925,
 'fruits': -1.3014528038900453,
 'soft': -1.2966723116527357,
 'blend': -1.2778621306119553,
 'crisp': -1.27202253744395,
 'blackberry': -1.2693385485544593,
 'sweet': -1.2681234193696043,
 'apple': -1.2657976192105747,
 'texture': -1.2657593873797561,
 'offers': -1.2588222439883143,
 'white': -1.2431588930450679,
 'shows': -1.237197616495104,
 'light': -1.2263655314931408,
 'dark': -1.2214819825653094,
 'citrus': -1.215513962908123,
 'bright': -1.196563752546095,
 'well': -1.192011405049656,
 'cabernet': -1.1903316981702914,
 'vanilla': -1.189876605933423,
 'full': -1.1690787148885078,
 'juicy': -1.158342907667301,
 'pepper': -1

In [34]:
tfIdf = {}
for var in varietyDict:
    temp = {}
    for word in tfDict[var].keys():
        temp[word] = tfDict[var][word] * idfDict[word]
    tfIdf[var] = temp
tfIdf["Pinot Gris"]

{'pear': -0.09728548751488283,
 'pinot': -0.06292871404718263,
 'gris': 0.018881977986931593,
 'apple': -0.040559008221183905,
 'dry': -0.028872744638538655,
 'fresh': -0.027423282272766903,
 'notes': -0.021579714742719084,
 'citrus': -0.024627432535695316,
 'nose': -0.019072199909834248,
 'rich': -0.017519022676431458,
 'white': -0.01994495859193144,
 'light': -0.019741955249600398,
 'crisp': -0.018323129227325208,
 'freshness': -0.03260362205675654,
 'spice': -0.013400536803231701,
 'melon': -0.0321547472525483,
 'lemon': -0.022281197412685347,
 'peach': -0.02105503214548708,
 'texture': -0.015711952955413822,
 'sweet': -0.01554481032411738,
 'clean': -0.024568913994768116,
 'touch': -0.01778481212094201,
 'full': -0.015044237752084846,
 'spicy': -0.018011502455835153,
 'balanced': -0.01701474937625425,
 'sweetness': -0.026499419824856853,
 'juicy': -0.013581472649086733,
 'green': -0.014344511450244088,
 'grapefruit': -0.020756291032421607,
 'style': -0.01784548031357905,
 'fruity':

In [35]:
#controlla bene i calcoli, probabilmente ci sono trooooppe parole che non centrano un cazzo tipo 
tfIdf["Espadeiro"]

{'apéritif': 9.337389624490935e-06,
 'bright': -0.00012353538638716654,
 'clean': -0.00018199195551680085,
 'crisp': -0.00011038029655015185,
 'flavor': -0.0001734692505253973,
 'full': -0.00012858322865029782,
 'lightly': -0.00022576943836065765,
 'pink': -0.00024451738049659433,
 'screwcap': -0.00013088157318117972,
 'strawberry': -0.0002114228398571717,
 'sweet': -0.00011103435945798129}

In [ ]:
#devi applicare stemming e lemmatisation e fare le stesse cose di qui sopra